<a href="https://colab.research.google.com/github/kolattukudy/jupyternotebooks/blob/master/arine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/MyDrive/adata"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
claims	enrollment.csv	ndc9_lookup.json  patient_ID29057550656.json


In [230]:
import pandas as pd

In [231]:
#How many patients were enrolled in the program as of July 1st, 2020?

e_df = pd.read_csv("/content/drive/MyDrive/adata/enrollment.csv")
min_date=min(e_df['enrollment_start_date'])
filter_by_date = e_df[(e_df['enrollment_end_date'] > min_date) & (e_df['enrollment_end_date'] < '2020-07-01')]
filter_by_date ['card_id'].nunique()

2106

In [232]:
!ls "/content/drive/MyDrive/adata/claims"

pharmacy_claims_2020-03-11.csv	pharmacy_claims_2020-06-05.csv
pharmacy_claims_2020-04-30.csv	pharmacy_claims_2020-07-03.csv
pharmacy_claims_2020-05-15.csv


In [233]:
#Step 2 - Load the pharmacy claims
import glob
files = glob.glob("/content/drive/MyDrive/adata/claims/*.csv")
claims_df = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f)
    claims_df = claims_df.append(csv)

#claims_df.head(5)

In [234]:
#number of rows
claims_df.size


299624

In [235]:
#Step 3 - Clean and prepare pharmacy claims
claims_df = claims_df[['allowed_amount','card_id','claim_number','claim_line_number','fill_date','ndc_11','claim_status','days_supply']]



In [236]:

#filter out denied claims and group 
status = ['PAID','REVERSAL']
claims_f_df=claims_df[claims_df.claim_status.isin(status)]

claim_gp_df=claims_f_df.groupby(['card_id','claim_number','ndc_11','fill_date'])['days_supply'].agg(['sum'])
#claim_gp_df.head(5)

In [237]:
claim_reset_df=claim_gp_df.reset_index()
claim_reset_df.size

19650

In [238]:

#Question 3: How many prepared claims do you have at the end of step 3?
claim_reset_df.head(5)
claim_gpf_df = claim_reset_df[claim_reset_df['sum'] != 0]
claim_gpf_df.size

19440

In [239]:
claims_merge_df=claims_f_df.merge(claim_gpf_df, how='inner', on=['card_id','claim_number','ndc_11','fill_date'])


In [240]:

#Step 4: Find the total days supply and amount allowed in the specified date range
!ls "/content/drive/MyDrive/adata/"

claims	enrollment.csv	ndc9_lookup.json  patient_ID29057550656.json


In [241]:
#load NDC data
ndc_df=pd.read_json("/content/drive/MyDrive/adata/ndc9_lookup.json")
tdf=ndc_df.transpose()
tdf['ndc9'] = tdf['ndc9'].replace('-', '', regex=True).astype(str)



In [242]:
claims_merge_df['ndc9'] = claims_merge_df['ndc_11'].astype(str).str[:-2].astype(str)


In [243]:
#Question 4: What is the highest amount_allowed? Which patient ?
high_amount=claims_merge_df[claims_merge_df['allowed_amount']==claims_merge_df['allowed_amount'].max()]
high_amount[['allowed_amount','card_id','ndc9']]

,allowed_amount,card_id,ndc9
3363,17660.53,82554696297,504190171
3364,17660.53,47290249924,504190171
3365,17660.53,27400762376,504190171


In [244]:
#Question 4: What is the highest amount_allowed? Which patient and generic drug does it correspond to?
generic_df=tdf[tdf['ndc9']=='504190171']
generic_df[['id','genericName']]


,id,genericName
50419-0171,504190171,regorafenib


In [245]:
#remove ID prefix
e_df['card_id'] = e_df['card_id'].astype(str).str[2:].astype(int)
#e_df.head(5)

In [246]:
#Question 5: How many unique generic names for the patient Abe Lincoln?

abe_df=e_df[(e_df['first_name'] == 'Abe') & (e_df['last_name'] == 'Lincoln')]
abe_df


,birth_date,card_id,enrollment_end_date,enrollment_start_date,first_name,gender,last_name
2886,1961-03-05,62402817684,2999-12-31,2020-01-01,Abe,M,Lincoln


In [247]:
#a. Only medication “fills” that occurred ​between Jan 1st, 2020 and June 30th, 2020 should be included in the summary
claims_merge_df = claims_merge_df[claims_merge_df['fill_date'].notna()]
claims_merge_df = claims_merge_df[(claims_merge_df['fill_date'] > '2020-01-01') & (claims_merge_df['fill_date'] < '2020-06-30')]

In [248]:
#patient summary prep
p_merge_df = e_df.merge(claims_merge_df, how = 'inner', on='card_id')
tdf=tdf[['ndc9',	'genericName']]
p_data= p_merge_df[['card_id','first_name','last_name','allowed_amount','days_supply','sum','ndc9']]
generic =pd.merge(p_data, tdf, left_on='ndc9', right_on='ndc9')
#group_df=generic.groupby(['card_id'],as_index=True)
#p_summary=group_df[['card_id','first_name','last_name','allowed_amount','days_supply','sum','genericName']]



In [249]:
details = generic.groupby(['first_name', 'last_name','genericName']).apply(
    lambda r: r[['allowed_amount', 'days_supply']].to_dict(orient='records'))
details_df=details.to_frame()
details_df.head(5)


0
first_name last_name genericName                                                               
Abe        Lincoln   lisinopril               [{'allowed_amount': 10.0, 'days_supply': 90}, ...
                     metformin hydrochloride  [{'allowed_amount': 10.0, 'days_supply': 90}, ...
                     nitrofurantoin           [{'allowed_amount': 18.27, 'days_supply': 7}, ...
                     semaglutide              [{'allowed_amount': 802.32, 'days_supply': 28}...
Betsy      Ross      insulin glargine            [{'allowed_amount': 79.71, 'days_supply': 30}]

In [250]:
#The med_summary should be a json with the following properties:
#b. There should be a key for each unique generic medication name in the relevant time period
#c. Each generic medication name should have two aggregates included in a json: a key, value pair for “allowed_amount” and “days_supply”
#i. The allowed_amount should be the total of all the allowed_amount values for the relevant valid claims
#ii. The days_supply should also similarly be the total of all the days_supply for the relevant valid claims


def json_format(df_details):

    person_summary = {
              person_summary['card_id']:{**person_summary,'med_summary':{}} 
              for _,person_summary in df_details.iterrows()
              }
    for _,row in df_details.iterrows():
        person_summary[row['card_id']]['med_summary'][row['genericName']] = {
                                                        'days_supply': row['days_supply'],
                                                        'allowed_amount': row['allowed_amount']}
    person_summary = list(person_summary.values())
    # Cleaning up
    for attribute in person_summary:
        for key in ['ndc9','days_supply','allowed_amount','genericName','sum']:
            if key in attribute:
                del attribute[key]
    return  person_summary




In [251]:
#Step 5: Create a JSON summary file for each patient enrolled on July 1st, 2020 that we have pharmacy data for.

#1. Each patient should have their own individual JSON file called ‘patient_<id_number>.json’
#2. The json should contain the fields: “first_name”, “last_name” and “med_summary”
import json  
summry_list=json_format(generic)
for idx,element in enumerate (summry_list):
  textfile = open("patient_ID"+str(element["card_id"])+".json", "w")
  if "card_id" in element:
                del element["card_id"]
  textfile.write(json.dumps(element, indent = 4) )
  textfile.close()


In [252]:
p_no_dup=p_merge_df.drop_duplicates()
abe_df=p_no_dup[p_no_dup['card_id'] == 62402817684]
tdf.head(2)

,ndc9,genericName
51672-4025,516724025,amiodarone hydrochloride
68382-0773,683820773,ezetimibe


In [253]:
# Generics for Abe
ndc_list=abe_df['ndc9'].tolist()
abe_gen_df=tdf[tdf['ndc9'].isin(ndc_list)]
abe_gen_df['genericName']


68645-0544    metformin hydrochloride
47781-0308             nitrofurantoin
68645-0550                 lisinopril
00169-4132                semaglutide
Name: genericName, dtype: object

In [254]:
#How many unique generic names for the patient Abe Lincoln?
len(set(abe_gen_df['genericName']))

4